In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff

import dash
from jupyter_dash import JupyterDash
from dash import dcc, dash_table, html
from dash.dependencies import Input, Output

import numpy as np
import pandas as pd
import io

import ipywidgets as widgets
import os, math, time, pickle, subprocess
from os.path import isfile, join
from IPython.display import display, HTML, Image
from os import listdir

In [ ]:
JupyterDash.infer_jupyter_proxy_config()

In [ ]:
%load_ext watermark
%watermark -v -m -p pandas,numpy,plotly,dash,ipywidgets,jupyter_dash 

In [ ]:
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# colocar o caminho onde está os arquivos com os resultados para serem concatenados juntos

caminho = '/home/pedro/Documentos/Projeto HPV/Compilado_graficos'
files = listdir(caminho)
dfs = range(len(listdir(caminho)))
frames=[]
for i, j  in zip(dfs, files):
    i = pd.read_csv(os.path.join(caminho, j))
    i = i.drop(i.columns[[0]], axis=1)
    frames.append(i)

In [ ]:
uploader = widgets.FileUpload(
    accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=True  # True to accept multiple files upload else False
)
display(uploader)

In [ ]:
dfs = range((len(uploader.value.items())))
file = uploader.value.items()
frames=[]
for i, elem  in zip(dfs, uploader.value.items()):
    name, file_info = elem    
    content = file_info['content']
    content = io.StringIO(content.decode('utf-8'))
    i = pd.read_csv(content)
    i = i.drop(i.columns[[0]], axis=1)
    frames.append(i)

In [ ]:
df = pd.concat(frames).reset_index().drop(columns=['index'])
df.head()

In [ ]:
# Build App

label = 'HPV'
app = JupyterDash(__name__)
# app = dash.Dash(__name__)


app.layout = html.Div([
    html.H1("MTT app"),
    html.Label([
        label,
        dcc.Dropdown(id='dropdown', clearable=False, options=[{'label': c, 'value': c}
                for c in list(df[label].unique())],value='HPV-16')]),
    html.Div([
    dash_table.DataTable(
        id='table',
        columns=[{"name": i, "id": i} 
                 for i in df.columns],
        style_cell=dict(font_size= '11px',textAlign='center'),
        page_action="native",
        page_current= 0,
        style_header=dict(backgroundColor="lightgrey"),
        style_data=dict(backgroundColor="white"))]),html.Br(),html.Div([
        html.Div([
            dcc.Graph(id='graph'),
        ],className='six columns'),

        html.Div([
            dcc.Graph(id='regression'),
        ],className='six columns'),

    ],className='row')])
#---------------------------------------------------------------------------------------------------------------------------------------

# Define callback to update graph
@app.callback(Output(component_id='table', component_property='data'),
              Output(component_id='graph', component_property='figure'),
              Output(component_id='regression', component_property='figure'),
              Input(component_id="dropdown", component_property="value"))

#---------------------------------------------------------------------------------------------------------------------------------------

def update_figure(value):
    result = df.query('HPV == @value')
    table = result.to_dict('records')
    cor_controle_pos='rgb(128,177,211)'
    cor_controle_neg='rgb(188,128,189)'
    cor_tratamento='rgb(190,186,218)'
    fig1 = px.strip(result, x='Diluição', 
                y= ['B','C','D','E','F','G'], 
                color = ['DMSO 10%', 'PBS', '10 ug/ml', '20 ug/ml', '40 ug/ml','80 ug/ml','160 ug/ml', '320 ug/ml','640 ug/ml','1280 ug/ml'],
                color_discrete_map={'DMSO 10%':cor_controle_neg,'PBS': cor_controle_pos,'10 ug/ml':cor_tratamento,
                '20 ug/ml':cor_tratamento,'40 ug/ml':cor_tratamento,'80 ug/ml':cor_tratamento,'160 ug/ml':cor_tratamento,
                '320 ug/ml':cor_tratamento,'640 ug/ml':cor_tratamento,'1280 ug/ml':cor_tratamento}, 
                   labels={"Diluição": "Tratamento",  "value": "Viabilidade (%)","Diluição": "Tratamento"},stripmode='overlay',template="none").update_traces(
                                                                           jitter = 1,
                                                                           opacity=0.6,
                                                                           marker_size=8,
                                                                           marker_line_width=1,
                                                                           marker_line_color='rgba(0,0,0,0.8)',
                                                                           #marker_color='rgba(0,0,0,0.8)',
                                                                           showlegend=True,hovertemplate = "Treatment: %{x}<br>" + "Viability: %{y}<extra></extra>")

    fig1.update_layout(title="Peptide"+' '+result[label].unique()[0],xaxis_title="Treatment",yaxis_title="Viability (%)",template='none',width=1000,height=600,
                      legend=dict(x=1, y=1,title_text='',traceorder="normal",bordercolor="Black",borderwidth=0,font=dict(size=10, color="black")))

    n_controle = 2
    color_trat = [cor_tratamento]*(len(result.loc[:,('Diluição')])-n_controle)
    colors = [cor_controle_neg, cor_controle_pos]+color_trat
    fig1.add_bar(name='Bar',x=result['Diluição'], y=result['Média'], marker_color=colors,error_y=dict(type ='data',array = result['interval'],
                visible = True,thickness=1.5, width=5),showlegend=True,
                customdata=result.loc[:,('CI', 'n')],hovertemplate = "Treatment: %{x}<br>" + 
                                                                     "Viability ± CI: %{y}<br>" +
                                                                     "Confidence Interval: %{customdata[0]}<br>"+
                                                                     "n: %{customdata[1]} <extra></extra>")

    for m,mm in result.iterrows():
        if mm.significance_PBS == '***':
            fig1.add_annotation(text="***",name="p-value", x=mm.Diluição, y=mm.Média+mm.interval+15, showarrow=False,font=dict(size=12, color="black"),)
        elif mm.significance_PBS == '**':
            fig1.add_annotation(text="**",name="p-value", x=mm.Diluição, y=mm.Média+mm.interval+15, showarrow=False,font=dict(size=12, color="black"),)
        elif mm.significance_PBS == '*':
            fig1.add_annotation(text="*",name="p-value", x=mm.Diluição, y=mm.Média+mm.interval+15, showarrow=False,font=dict(size=12, color="black"),)
        elif mm.significance_PBS == 'ns':
            pass
        elif mm.significance_PBS == np.nan:
            pass

    fig1.add_annotation (text = '<sup>Data points shown on the bar graph represent individual viability value. 𝙸 bars represent 95% confidence intervals. Statistical anlysis were calculated using ANOVA with Tukey’s multiple comparisons test.<br>' 
                               'Statistical differences are shown as * (p<0.05), ** (p<0.01), *** (p<0.001) compared with control PBS. <br>',
                        xref = "x domain",
                        yref = "y domain",
                        x = 0,
                        y = -0.16,
                        showarrow = False,
                        font = dict(size = 14),
                        align = "left")

    fig1.update_xaxes(title_standoff = 12, ticklabelposition='inside', ticks='')
    
    # Regression plot
    
    reg_table = result.loc[:, ('Diluição','B','C','D','E','F','G')].T
    reg_table.columns = reg_table.iloc[0]
    reg_table = reg_table.drop(['Diluição'])
    
    # Handle table to plot
    
    df_melt = pd.melt(reg_table.reset_index(), id_vars=['index'])
    df_melt.columns = ['index', 'tratamento', 'viabilidade']
    df_melt = df_melt.drop(columns=['index'])
    
    if any(df_melt['viabilidade'].isnull().values) == True:
        df_melt = df_melt.dropna(subset = ["viabilidade"], inplace=False).reset_index().drop(columns=['index'])
    else:
        pass
    
    df_melt['tratamento'] = df_melt['tratamento'].map({'10 ug/ml':10,'20 ug/ml':20,'40 ug/ml':40,'80 ug/ml':80,'160 ug/ml':160,'320 ug/ml':320,'640 ug/ml':640,
                                                  '1280 ug/ml':1280})
    if any(df_melt['tratamento'].isnull().values) == True:
        df_melt = df_melt.dropna(subset = ["tratamento"], inplace=False)
    else:
        pass
    
    df_melt = df_melt.rename(columns={"tratamento": 'diluição'})
    
    
    fig_reg = px.scatter(df_melt, x='diluição', y='viabilidade',opacity=0.65, trendline='ols', trendline_color_override='darkblue')
    fig_reg.update_layout(title="Peptide"+' '+result[label].unique()[0],xaxis_title="Concentration (ug/ml)",yaxis_title="Viability (%)",template='none',width=1000,height=500,
                      legend=dict(x=1, y=1,title_text='',traceorder="normal",bordercolor="Black",borderwidth=0,font=dict(size=10, color="black")))
        
    return (table,fig1,fig_reg)
    
    
# Run app and display result inline in the notebook
# app.run_server(mode='inline')
if __name__ == '__main__':
    app.run_server(debug=True)